In [26]:
using SDDP, HiGHS


Ω = [
    (;load = Dict(1 => [100,130,100], 
                 2 =>  [200,140,200],
                 3 =>  [100,210,150])),
                 
    (;load = Dict(1 => [200,230,150], 
                 2 =>  [300,330,200],
                 3 =>  [200,100,300])),
                 
    (;load = Dict(1 => [300,400,200], 
                 2 =>  [400,100,200],
                 3 =>  [300,500,400]))
]

OmegaRow = [1,2,3]
OmegaBus = [1,2,3]
OmegaT   = [1,2,3]
RampT    = [2,3]

branch = Dict(1 => (1, 2), 2 => (1, 3), 3 => (2,3))

line_cost = Dict(1 => 10, 2 => 10, 3 => 10)
B_ij = Dict(1 => 0.40, 2 => 0.40, 3 => 0.40) 
S_ij = Dict(1 => 100, 2 => 100, 3 => 100) 
n0 = Dict(1 => 1, 2 => 1, 3 => 1)

p_max  = Dict(1 => 100, 2 =>100, 3 => 100)
p_ramp = Dict(1 => 100, 2 =>100, 3 => 100)

probability = [0.3,0.4,0.3]

model = SDDP.MarkovianPolicyGraph(;
    transition_matrices = Array{Float64,2}[
        [1.0]',
        [0.3 0.4 0.3],
        [0.3 0.4 0.3 ; 0.3 0.4 0.3; 0.3 0.4 0.3],
    ],
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do subproblem, node
    t, markov_state = node

    @variable(subproblem, 0 <= i_coal[OmegaBus], SDDP.State, initial_value = 0)
    @variable(subproblem, 0 <= x_line[OmegaRow] <=1, SDDP.State, initial_value = 0, Int)

    @variable(subproblem, 0 <= i_coal_loc[OmegaBus])
    @variable(subproblem, 0 <= x_line_loc[OmegaRow] <=1, Int)

    @variables(subproblem, begin
        p_coal[OmegaBus, OmegaT] >= 0
        p_exist[OmegaBus, OmegaT] >= 0
        p_shed[OmegaBus, OmegaT] >= 0
        flow[OmegaRow, OmegaT] 
        theta[OmegaBus, OmegaT]
        Load[OmegaBus, OmegaT]
    end)

    @constraint(subproblem,  adacoal[n=OmegaBus], i_coal[n].out == i_coal[n].in + i_coal_loc[n] )
    @constraint(subproblem,  adaline[l=OmegaRow], x_line[l].out == x_line[l].in + x_line_loc[l] )

    @constraint(subproblem,  nodbal[n=OmegaBus, t=OmegaT], sum(flow[l,t] for l in OmegaRow if branch[l][2] == n) - sum(flow[l,t] for l in OmegaRow if branch[l][1] == n)
    + p_coal[n,t] + p_exist[n,t] + p_shed[n,t] - Load[n,t] == 0 )
    @constraint(subproblem,  pflow[l=OmegaRow, t=OmegaT], flow[l,t] == (1/B_ij[l])*(theta[branch[l][1],t] - theta[branch[l][2],t]) )
    @constraint(subproblem,  fcap1[l=OmegaRow, t=OmegaT], flow[l,t] - S_ij[l]*(n0[l] + x_line[l].in) <= 0 )
    @constraint(subproblem,  fcap2[l=OmegaRow, t=OmegaT],- flow[l,t] - S_ij[l]*(n0[l] + x_line[l].in) <= 0 )
    @constraint(subproblem,  maxp1[n=OmegaBus, t=OmegaT], p_exist[n,t] <= p_max[n] )
    @constraint(subproblem,  maxc1[n=OmegaBus, t=OmegaT], p_coal[n,t]  <= 100*i_coal[n].in )
    @constraint(subproblem,  pshed[n=OmegaBus, t=OmegaT], p_shed[n,t] <= Load[n,t] )


    @constraint(subproblem,  ramp1[n=OmegaBus, t=RampT], p_coal[n,t] - p_coal[n,t-1] <=  20)
    @constraint(subproblem,  ramp2[n=OmegaBus, t=RampT], p_coal[n,t] - p_coal[n,t-1] >= -20)
    @constraint(subproblem,  ramp3[n=OmegaBus, t=RampT], p_exist[n,t] - p_exist[n,t-1] <= 20)
    @constraint(subproblem,  ramp4[n=OmegaBus, t=RampT], p_exist[n,t] - p_exist[n,t-1] >= -20)

    probability = [0.3,0.4,0.3]

    SDDP.parameterize(subproblem, Ω, probability) do ω
        for n in OmegaBus for t in OmegaT
            JuMP.fix(Load[n,t], ω.load[n][t])
        end end
        @stageobjective(
            subproblem,
            sum(50*i_coal_loc[n] for n in OmegaBus) + sum(line_cost[l]*x_line_loc[l] for l in OmegaRow)
          + sum(5*p_exist[n,t] + 2*p_coal[n,t] + 10*p_shed[n,t] for n in OmegaBus for t in OmegaT)
        )
    end
end



A policy graph with 7 nodes.
 Node indices: (1, 1), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)


In [27]:
SDDP.train(model, log_frequency= 2)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-24
-------------------------------------------------------------------
problem
  nodes           : 7
  state variables : 6
  scenarios       : 2.43000e+02
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [73, 73]
  AffExpr in MOI.EqualTo{Float64}         : [24, 24]
  AffExpr in MOI.GreaterThan{Float64}     : [12, 12]
  AffExpr in MOI.LessThan{Float64}        : [57, 57]
  VariableRef in MOI.GreaterThan{Float64} : [40, 40]
  VariableRef in MOI.Integer              : [6, 6]
  VariableRef in MOI.LessThan{Float64}    : [6, 7]
numerical stability report
  matrix range     [1e+00, 1e+02]
  objective range  [1e+00, 5e+01]
  bounds range     [1e+00, 1e+00]
  rhs range        [2e+01, 1e+02]
--------------------

In [7]:
print(model[(1,1)].subproblem)

Min 50 i_coal_loc[1] + 50 i_coal_loc[2] + 50 i_coal_loc[3] + 10 x_line_loc[1] + 10 x_line_loc[2] + 10 x_line_loc[3] + 5 p_exist[1] + 2 p_coal[1] + 10 p_shed[1] + 5 p_exist[2] + 2 p_coal[2] + 10 p_shed[2] + 5 p_exist[3] + 2 p_coal[3] + 10 p_shed[3] + _[37]
Subject to
 adacoal[1] : -i_coal[1]_in + i_coal[1]_out - i_coal_loc[1] = 0.0
 adacoal[2] : -i_coal[2]_in + i_coal[2]_out - i_coal_loc[2] = 0.0
 adacoal[3] : -i_coal[3]_in + i_coal[3]_out - i_coal_loc[3] = 0.0
 adaline[1] : -x_line[1]_in + x_line[1]_out - x_line_loc[1] = 0.0
 adaline[2] : -x_line[2]_in + x_line[2]_out - x_line_loc[2] = 0.0
 adaline[3] : -x_line[3]_in + x_line[3]_out - x_line_loc[3] = 0.0
 nodbal[1] : p_coal[1] + p_exist[1] + p_shed[1] - flow[1] - flow[2] - Load[1] = 0.0
 nodbal[2] : p_coal[2] + p_exist[2] + p_shed[2] + flow[1] - flow[3] - Load[2] = 0.0
 nodbal[3] : p_coal[3] + p_exist[3] + p_shed[3] + flow[2] + flow[3] - Load[3] = 0.0
 pflow[1] : flow[1] - 2.5 theta[1] + 2.5 theta[2] = 0.0
 pflow[2] : flow[2] - 2.5 the